Zapoznaj się z prezentacją `lab-03-pl-slides.pdf` i zaimplementuj algorytm SGD dla przedstawionej tam sieci w wersji batch size = 1.

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from copy import deepcopy

rng = np.random.default_rng(2023)

# Inicjalizacja

* funkcja straty $L=CE$,
* $\sigma$ oraz $\sigma'$,
* losowe wagi: $W$, $b_1$, $U$, $b_2$.

In [2]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)

In [3]:
def sigmoid(x):
    return (np.exp(x)) / (1 + np.exp(x))

def sigmoid_prime(x):
    return sigmoid(x) * ( 1 - sigmoid(x) )

vsigmoid = np.vectorize(sigmoid)
vsigmoid_prime = np.vectorize(sigmoid_prime)

Losuję wagi, uwaga na kształty związane ściśle z architekturą sieci typu *fully connected* (input 3 neurony, warstaw ukryta 4 neurony, output 1 neuron).

In [4]:
W = rng.uniform(size=(4, 3))
b1 = rng.uniform(size=(4, 1))
U = rng.uniform(size=(1, 4))
b2 = rng.uniform()

Twarde kopie [(shallow/hard copy in Python)](https://docs.python.org/3/library/copy.html) potrzebne do inicjalizacji modelu `Keras` z tymi samymi wylosowanymi wagami.

In [5]:
Wc = deepcopy(W)
b1c = deepcopy(b1)
Uc = deepcopy(U)
b2c = deepcopy(b2)

# algorytm Stochastic Gradient Descent - przypomnienie skrótowe dla pojedynczej macierzy wag $W$.

(dla pojedynczego *batch'a*)

1. zainicjuj wagi $W_0$,
2. oblicz ryzyko $L(W_0)$ na *batch'u*,
3. oblicz `grad` = $\nabla_W L(W_0)$ na *batch'u* (zastosuj operację uśredniania jeżeli \#batch > 1),
4. $W_1 = W_0 - \eta \cdot$ `grad`,
5. weź kolejny batch i wróć do kroku 2.

# Zadanie 1

Napisz funkcję `forward`, która implementuje krok 2 dla zadanej sieci.

In [6]:
def forward(
    x, # data
    W, # weights 1st layer
    b1, # bias 1st layer
    U, # weights 2nd layer
    b2, # weights 2nd layer
    y # true response
):
    z = W @ x + b1
    h = vsigmoid(z).reshape(-1, 1)    # uwaga na ksztalty
    teta = U @ h + b2
    y_hat = vsigmoid(teta)
    risk = bce(y_true=y, y_pred=y_hat)

    return z, h, teta, y_hat, risk

# Zadanie 2

Napisz funkcję `backward`, która implementuje krok 3 dla zadanej sieci.

In [7]:
def backward(
    y_hat,
    y,
    U,
    h,
    x,
    z
):
    """
    Nawiasem mówiąc: to jest docstring: metoda dokumentowania funkcji
    (zob. https://peps.python.org/pep-0257/)

    Funkcja `backward` implementuje mechanizm back-propagation,
    propagujemy blad wstecz przez siec az do wag.

    Komentarze przy poszczegolnych linijkach referuja wprost do wzorow ze
    slajdow. Po to wyprowadzalisimy analitycznie te wzory, zeby teraz
    je zaimplementowac w numpy.
    """
    delta_1 = y_hat - y    # wzor 3 ze slajdów
    delta_2 = delta_1 * U * vsigmoid_prime(z).reshape(1, -1)    # wzor 5 ze slajdów

    grad_U = delta_1 * h.reshape(1, -1)    # wzor 7 ze slajdow
    grad_b2 = delta_1    # wzor 8 ze slajdow

    grad_W = np.outer(delta_2.T, x.T)    # wzor 6 ze slajdow
    grad_b1 = delta_2.T    # wzor 9 ze slajdow

    return grad_W, grad_b1, grad_U, grad_b2

# Symulacja danych

In [8]:
x = rng.normal(0, 1, size=(100, 3))    # warstwa input w naszej sieci ma 3 neurony
y = rng.binomial(1, 0.5, size=(100, 1))    # siec typu binary classification

# Zadanie 4

Utwórz listę `risks`. Napisz pętlę, która:

1. wykona krok `forward`,
2. wykona krok `backward`,
3. zaktualizuje wagi zgodnie z SGD,
4. dołączy do listy `risk` wyliczone ryzyko dla danego batch'a
(chcemy śledzić rezultaty dla poszczególnych batch'y).

Użyj stałej $\eta = 0.1$.

Wykonaj jedną epoch, niech \#batch (czyli rozmiar batch'a) = 1.
Zazwyczaj nie wykonujemy back-propagacji dla tak małych batch'y, robimy to jedynie w celach nauczenia się tego, jak funkcjonuje prosta sieć z jedną warstwą ukrytą.

**Zauważmy, że powyższy krok to de facto implementacja sieci neuronowej**.

Znamy architekturę sieci, trenujemy wagi - jak je wytrenujemy po zadanej liczbie epochs (1), to mamy wytrenowane wagi $\hat{W}$, $\hat{b_1}$, $\hat{U}$, $\hat{b_2}$ i możemy dokonywać predykcji.



In [9]:
risks = []
for idx in range(x.shape[0]):
    z, h, teta, y_hat, risk = forward(x[idx, :].reshape(-1, 1), W, b1, U, b2, y[idx, :].reshape(1, 1))
    grad_W, grad_b1, grad_U, grad_b2 = backward(y_hat, y[idx, :], U, h, x[idx, :].reshape(-1, 1), z)
    W = W - 0.1*grad_W
    b1 = b1 - 0.1*grad_b1
    U = U - 0.1*grad_U
    b2 = b2 - 0.1*grad_b2
    risks.append(risk.numpy())

In [10]:
len(risks)

100

Lista `risks` zawiera ryzyka dla danych batch'y.
Ponieważ \#batch=1 (1 obserwacja = 1 batch), to długość listy = 100.

Nawiasem mówiąc, poprawniej byłoby mówić o stracie (*loss*).
Te dwa pojęcia często używane są jednak zamiennie, przyzwyczajajcie się Państwo do tego - tym bardziej, że zazwyczaj jednak \#batch > 1.

# Porównanie z odpowiednim modelem w `Keras`

Państwo widzicie pseudo-kod, implementujący analogiczną sieć neuronową za pomocą biblioteki `Keras`. Nie udostępniam pełnego kodu, ponieważ odtworzenie sieci w `Keras` będzie jednym z podpunktów mini-projektu 1.

```python
model_keras = tf.keras.Sequential()

model_keras.add(...)
model_keras.add(...)
model_keras.add(...)

model_keras.compile(...)
```

In [12]:
risks_keras = list()
for idx in range(x.shape[0]):
    history = model_keras.train_on_batch(
        x[idx, :].reshape(1, -1), 
        y[idx, :].reshape(1, 1), 
        return_dict=True
    )
    risks_keras.append(history.get('loss'))

# Porównanie wyników modelu `Keras` oraz naszej implementacji

Porównujemy obliczone ryzyka dla każdego batcha (czyli dla każdej obserwacji, gdyż \#batch = 1 w naszym zadaniu).

Empirycznie pokazujemy identyczność naszej implementacji w Python i odpowiedniego modelu stworzonego za pomocą `Keras`.

In [13]:
pd.DataFrame({"risk": risks, "risks_keras": risks_keras, "y":y.flatten()})

,risk,risks_keras,y
0,2.298263,2.298264,0
1,0.132735,0.132735,1
2,2.121013,2.121013,0
3,0.153239,0.153239,1
4,1.697382,1.697383,0
...,...,...,...
95,0.728581,0.728581,0
96,0.737579,0.737579,1
97,0.746102,0.746102,0
98,0.687447,0.687447,0


Porównajmy wytrenowaną macierz wag $W$:

In [14]:
W    # z implementacji Python

array([[0.06830175, 0.20564328, 0.10226974],
       [0.43391805, 0.71338097, 0.55838187],
       [0.74219693, 0.61207383, 0.90912334],
       [0.15076252, 0.49091113, 0.83541364]])

In [15]:
model_keras.layers[0].get_weights()[0].T    # z modelu Keras

array([[0.06830178, 0.2056433 , 0.10226975],
       [0.43391806, 0.7133812 , 0.5583821 ],
       [0.7421972 , 0.61207384, 0.9091235 ],
       [0.15076248, 0.49091095, 0.83541346]], dtype=float32)

---

# Dokonajmy predykcji dla obserwacji testowej

Niech $x_\text{test}$ = $(0.3, 0.5, -1.)^T$

In [16]:
model_keras.predict(np.array([[0.3, 0.5, -1.]]))

1/1 [==============================] - 0s 187ms/step


array([[0.43038067]], dtype=float32)

Dokonywanie predykcji z aktualnej implementacji sieci w Python jest niewygodne - nie mamy do tego żadnej specjalnej funkcji.

Jeżeli nie napiszemy żadnej funkcji, lub nie ubierzemy algorytmu SGD w klasę z metodami, to np. możemy dokonać predykcji licząć krok `forward`:

In [17]:
_, _, _, y_hat, _ = forward(np.array([[0.3], [0.5], [-1.]]), W, b1, U, b2, y[0, :].reshape(1, 1))

In [18]:
y_hat

array([[0.43038066]])

Predykcje są identyczne.

Uwaga Pythonowa: funkcja `forward` zwraca 5 zmiennych, nas interesuje zmienna na 4 pozycji. Muszę przypisać wyniki funkcji do 5 zmiennych.
Mogę przypisać zmienne na pozycjach 1, 2, 3, 5 do dummy variables `_` - nie będziemy z nich korzystać.

# Epochs i batches

Powyżej trenowaliśmy modele z ustawieniem batch size = 1 żeby sprawdzać krok po kroku zbieżność naszej implementacji i modelu `Keras`.

Projekt za aktywność skupi się na implementacji dowolnej liczby epochs i batches.

**Uwaga**: z \#batch > 1 będziemy już potrzebować uśredniać gradienty
$\frac{\partial L}{\partial U},
\frac{\partial L}{\partial b_2},
\frac{\partial L}{\partial W},
\frac{\partial L}{\partial b_1}$
dla poszczególnego batch'a.

Dla macierzy $\frac{\partial L}{\partial W}$ wygodnie będzie użyć 3-wymiarowych `array`.

Potrzebna będzie nam funkcja `mean_gradient(gradients)`, która zwraca średni gradient bez względu na kształt gradientów.

# Zadanie 5

Napisz funkcję `mean_gradient(gradients)`, która zwróci średni gradient dla listy gradientów podanych do funkcji niezależnie od ich kszałtu.

Argument `gradients` to *iterable* (np. lista lub krotka), które zawiera `arrays` o danym kształcie (x, y).
Funkcja `mean_gradient` powinna zwrócić średnią z zadanych `arrays` o kształcie (x, y).

```python
def mean_gradient(gradients):
    pass    # proszę spróbować zaimplementować we własnym zakresie
```

Przykładowe inputs oraz rezultaty zmiennej `mean_gradient`:

Działa dla macierzy:

In [20]:
przykladowa_lista_gradientow = [rng.uniform(size=(3,2)) for _ in range(5)]

In [21]:
przykladowa_lista_gradientow

[array([[0.10875289, 0.72862798],
        [0.76762435, 0.97706667],
        [0.46751619, 0.72357884]]), array([[0.35105743, 0.30917482],
        [0.33764007, 0.07326548],
        [0.34753594, 0.49679605]]), array([[0.24619359, 0.6717545 ],
        [0.88249399, 0.82245133],
        [0.09092259, 0.0558713 ]]), array([[0.06656914, 0.4371196 ],
        [0.27163903, 0.73940405],
        [0.79130863, 0.01788042]]), array([[0.0510878 , 0.82033552],
        [0.77539141, 0.9645174 ],
        [0.07849517, 0.38403727]])]

In [22]:
mean_gradient(przykladowa_lista_gradientow)

array([[0.16473217, 0.59340249],
       [0.60695777, 0.71534098],
       [0.35515571, 0.33563278]])

Działa dla skalarów:

In [23]:
przykladowa_lista_gradientow_2 = [rng.uniform() for _ in range(5)]

In [24]:
przykladowa_lista_gradientow_2

[0.5090372091011456,
 0.900869153260128,
 0.849403647531704,
 0.8986535370450214,
 0.1287272599667597]

In [25]:
mean_gradient(przykladowa_lista_gradientow_2)

0.6573381613809517

---